In [ ]:
import tensorflow as tf
import os
import cv2
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [ ]:
from tensorflow import keras
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/archive (13).zip", "r")
zip_ref.extractall("files")
zip_ref.close()

In [ ]:
data = keras.preprocessing.image_dataset_from_directory(
    directory="/content/files/images", label_mode=None, image_size=(64, 64), batch_size=32,
    shuffle=True
).map(lambda x: x/255.0)

Found 63565 files belonging to 1 classes.


In [ ]:
newpath = r'/content/results' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D,BatchNormalization,Input
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU

In [ ]:
latent_dim = 256
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [ ]:
discriminator = keras.Sequential(
    [
        keras.Input(shape=(64,64,3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ]
)


In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [ ]:
def discriminator_loss(fake_image,real_image):
  real_loss=cross_entropy(tf.ones_like(real_image),real_image)
  fake_loss=cross_entropy(tf.zeros_like(fake_image),fake_image)
  total_loss=real_loss+fake_loss
  return total_loss

In [ ]:
def generator_loss(fake_image):
  return cross_entropy(tf.ones_like(fake_image),fake_image)

In [ ]:
generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8192)              2105344   
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 128)       262272    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       524544    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 512)       20

In [ ]:
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        3136      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       131200    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         262272    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
from tqdm import tqdm

for epoch in range(20):
  for idx,real in enumerate(tqdm(data)):
    batch_size=real.shape[0]
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      random_latent_vectors = tf.random.normal(shape=(batch_size,latent_dim))
      fake_images=generator(random_latent_vectors,training=True)
      real_output=discriminator(real,training=True)
      fake_output=discriminator(fake_images,training=True)
      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss( fake_output,real_output)
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
      if idx%100 ==0:
        img = keras.preprocessing.image.array_to_img(fake_images[0])
        img.save("/content/results/generated_img_%03d_%d.png" % (epoch, idx))
    

 46%|████▌     | 915/1987 [05:17<06:11,  2.88it/s]